# INM427 Neural Computing Final Coursework
## By Ho Yin Tam

Import the library.

In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetClassifier 
import time
from joblib import dump, load

Import the training data and convert it into tensor datatype.

In [37]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')


X_train_tensor = torch.tensor(X_train.to_numpy()).float()

# Convert the dataframe of the target label into series
y_train = y_train.squeeze()
y_train_tensor = torch.tensor(y_train.to_numpy()).long()

# Display the shape and data type of the feature and target label
print('Shape of training data (features):', X_train_tensor.shape)
print('Shape of training data (target label):', y_train_tensor.shape)
print('Data type:', X_train_tensor.dtype)
print('Data type:', y_train_tensor.dtype)

Shape of training data (features): torch.Size([596, 11])
Shape of training data (target label): torch.Size([596])
Data type: torch.float32
Data type: torch.int64


Define a class for the multilayer perceptron with one hidden layer.

In [38]:
class MLP_onehiddenlayer(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(MLP_onehiddenlayer, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim = 1)
     
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x
    
MLP_clf = NeuralNetClassifier(MLP_onehiddenlayer, module__input_size = 11, module__output_size = 2)

Set up the grid search parameters of hidden size, momentum and learning rate.

In [39]:
# Number of neurons in hidden layer: 10, 100, 1000
# Momentum: 0.7, 0.8, 0.9 
# Learning rate: 0.001, 0.01, 0.1 

gs_parameters = {'module__hidden_size': [10, 100, 1000],
                 'optimizer__momentum': [0.7, 0.8, 0.9],
                 'lr': [0.001, 0.01, 0.1]}
gs_parameters

{'module__hidden_size': [10, 100, 1000],
 'optimizer__momentum': [0.7, 0.8, 0.9],
 'lr': [0.001, 0.01, 0.1]}

Initiate the multilayer perceptron and perform grid search with 10-fold cross validation.

In [40]:
# Perform grid search with 10-fold cross validation
grid_search = GridSearchCV(estimator = MLP_clf, param_grid = gs_parameters, cv = 10, refit = True)

# Train the model and count the training time
start_time = time.time()
grid_search.fit(X_train_tensor, y_train_tensor)
end_time = time.time()

# Display the best score and best parameters
print('Best score is:', grid_search.best_score_)
print('Best parameters is:', grid_search.best_params_)
print('Training time:', end_time - start_time)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7479       0.5370        0.7502  0.0289
      2        0.7472       0.5370        0.7492  0.0302
      3        0.7463       0.5370        0.7481  0.0282
      4        0.7453       0.5370        0.7469  0.0277
      5        0.7444       0.5370        0.7458  0.0279
      6        0.7434       0.5370        0.7447  0.0249
      7        0.7425       0.5370        0.7436  0.0311
      8        0.7416       0.5370        0.7426  0.0291
      9        0.7407       0.5370        0.7415  0.0248
     10        0.7398       0.5370        0.7405  0.0280
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7549       0.5093        0.7371  0.0275
      2        0.7535       0.5093        0.7353  0.0348
      3        0.7518       0.5185        0.7335  0.0287
      4        0.7499       0.5

      3        0.6676       0.5741        0.6530  0.0319
      4        0.6656       0.5741        0.6512  0.0295
      5        0.6636       0.5741        0.6495  0.0313
      6        0.6617       0.5833        0.6478  0.0809
      7        0.6599       0.5833        0.6462  0.0340
      8        0.6581       0.5833        0.6445  0.0299
      9        0.6563       0.5833        0.6430  0.0249
     10        0.6547       0.5833        0.6415  0.0292
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7345       0.5370        0.7354  0.0256
      2        0.7325       0.5370        0.7323  0.0237
      3        0.7296       0.5370        0.7287  0.0247
      4        0.7263       0.5370        0.7250  0.0270
      5        0.7230       0.5370        0.7214  0.0289
      6        0.7198       0.5370        0.7178  0.0310
      7        0.7167       0.5370        0.7144  0.0260
      8        0.7138       0.5

      6        0.7496       0.3056        0.7636  0.0720
      7        0.7467       0.3241        0.7603  0.0346
      8        0.7438       0.3241        0.7571  0.0309
      9        0.7411       0.3241        0.7541  0.0279
     10        0.7385       0.3519        0.7511  0.0314
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6770       0.5370        0.7001  0.0267
      2        0.6758       0.5370        0.6983  0.0269
      3        0.6740       0.5648        0.6961  0.0335
      4        0.6721       0.5833        0.6939  0.0318
      5        0.6701       0.5833        0.6917  0.0302
      6        0.6681       0.5833        0.6895  0.0284
      7        0.6661       0.5926        0.6873  0.0249
      8        0.6641       0.5926        0.6851  0.0290
      9        0.6622       0.6019        0.6830  0.0324
     10        0.6604       0.6019        0.6810  0.0294
  epoch    train_loss    valid_

      6        0.6421       0.7593        0.6324  0.0317
      7        0.6400       0.7593        0.6300  0.0331
      8        0.6379       0.7685        0.6276  0.0346
      9        0.6358       0.7778        0.6252  0.0287
     10        0.6336       0.7870        0.6229  0.0303
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6778       0.5185        0.6793  0.0315
      2        0.6769       0.5278        0.6780  0.0432
      3        0.6754       0.5833        0.6763  0.0327
      4        0.6735       0.6019        0.6742  0.0250
      5        0.6713       0.6389        0.6720  0.0269
      6        0.6690       0.6389        0.6697  0.0239
      7        0.6666       0.6574        0.6674  0.0285
      8        0.6642       0.6759        0.6650  0.0539
      9        0.6618       0.6944        0.6626  0.0317
     10        0.6593       0.7037        0.6603  0.0282
  epoch    train_loss    valid_

      5        0.6593       0.7685        0.6359  0.0321
      6        0.6530       0.7685        0.6298  0.0704
      7        0.6472       0.7593        0.6241  0.0349
      8        0.6416       0.7593        0.6186  0.0320
      9        0.6364       0.7500        0.6135  0.0331
     10        0.6315       0.7593        0.6086  0.0370
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7011       0.4815        0.7061  0.0313
      2        0.6913       0.4907        0.6935  0.0388
      3        0.6793       0.5556        0.6810  0.0406
      4        0.6679       0.5926        0.6696  0.0385
      5        0.6576       0.6481        0.6593  0.0310
      6        0.6484       0.6759        0.6499  0.0334
      7        0.6401       0.7037        0.6415  0.0308
      8        0.6326       0.7222        0.6337  0.0331
      9        0.6258       0.7407        0.6266  0.0349
     10        0.6195       0.7

...

      4        0.4137       0.8426        0.4099  0.0285
      5        0.3936       0.8333        0.4126  0.0289
      6        0.3817       0.8241        0.4044  0.0309
      7        0.3699       0.8333        0.3954  0.0361
      8        0.3607       0.8333        0.3911  0.0280
      9        0.3531       0.8333        0.3910  0.0306
     10        0.3468       0.8241        0.3919  0.0274
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6431       0.7685        0.4895  0.0229
      2        0.4851       0.8241        0.3939  0.0241
      3        0.4226       0.8148        0.4003  0.0193
      4        0.3942       0.8056        0.4193  0.0229
      5        0.3720       0.8056        0.4374  0.0275
      6        0.3666       0.8148        0.4344  0.0259
      7        0.3565       0.8241        0.4272  0.0301
      8        0.3500       0.8148        0.4214  0.0299
      9        0.3429       0.8

      3        0.4152       0.8426        0.4444  0.0306
      4        0.4151       0.8241        0.4594  0.0602
      5        0.3751       0.8333        0.4489  0.0291
      6        0.3527       0.8241        0.4163  0.0279
      7        0.3412       0.8333        0.3943  0.0303
      8        0.3313       0.8056        0.3997  0.0320
      9        0.3224       0.8148        0.4159  0.0290
     10        0.3101       0.8148        0.4292  0.0264
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6583       0.7963        0.4545  0.0270
      2        0.4802       0.8241        0.3702  0.0270
      3        0.4269       0.8148        0.3914  0.0269
      4        0.3995       0.7963        0.4300  0.0269
      5        0.3617       0.7778        0.4721  0.0264
      6        0.3638       0.7685        0.4629  0.0265
      7        0.3490       0.8148        0.4220  0.0233
      8        0.3327       0.8

      2        0.5590       0.7778        0.7214  0.0414
      3        0.5343       0.7500        0.8264  0.0343
      4        0.5762       0.7593        0.7726  0.0335
      5        0.5376       0.7500        0.6111  0.0319
      6        0.4834       0.7778        0.5140  0.0309
      7        0.4436       0.8148        0.4653  0.0420
      8        0.4143       0.8519        0.4442  0.0449
      9        0.3917       0.8611        0.4338  0.0303
     10        0.3727       0.8519        0.4286  0.0347
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.5808       0.7500        0.7512  0.0290
      2        0.6822       0.7315        0.9563  0.0291
      3        0.7305       0.7778        0.7398  0.0301
      4        0.5802       0.7778        0.5184  0.0320
      5        0.4678       0.8333        0.4761  0.0331
      6        0.4117       0.8519        0.4604  0.0299
      7        0.3714       0.8

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.9493       0.7778        0.7449  0.0273
      2        0.9159       0.8519        0.5402  0.0633
      3        0.5443       0.7870        0.6503  0.0432
      4        0.5198       0.8333        0.4743  0.0410
      5        0.4121       0.8333        0.4213  0.0304
      6        0.3460       0.8148        0.4484  0.0326
      7        0.3253       0.8426        0.4310  0.0304
      8        0.3110       0.8333        0.4495  0.0345
      9        0.3115       0.8148        0.4582  0.0322
     10        0.3119       0.8056        0.4575  0.0349
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7094       0.7685        1.1143  0.0294
      2        1.0230       0.8241        0.7058  0.0300
      3        0.8166       0.7500        0.8651  0.0322
      4        0.6280       0.8

      9        0.3754       0.8241        0.4036  0.0320
     10        0.3125       0.8611        0.4138  0.0326
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.5542       0.7500        0.7831  0.0329
      2        0.7476       0.7963        0.6924  0.0428
      3        0.7498       0.7037        1.0222  0.0368
      4        0.8132       0.8889        0.5018  0.0317
      5        0.6402       0.8426        0.6005  0.0328
      6        0.5932       0.7222        0.9012  0.0334
      7        0.5535       0.7963        0.6396  0.0289
      8        0.4754       0.8611        0.5076  0.0282
      9        0.4024       0.7778        0.5728  0.0279
     10        0.3323       0.7685        0.5692  0.0299
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7510       0.7593        1.0070  0.0329
      2        1.1448       0.8

Extract the mean score of all the grid search parameters. 

In [41]:
grid_search_result = pd.DataFrame(grid_search.cv_results_)
grid_search_result = grid_search_result[['param_module__hidden_size', 'param_lr', 'param_optimizer__momentum', 'mean_test_score']].sort_values(by = 'mean_test_score', ascending = False)
grid_search_result.columns = ['Number of neurons in hidden layer', 'Learning rate', 'Momentum', 'Score']
grid_search_result

,Number of neurons in hidden layer,Learning rate,Momentum,Score
22,100,0.1,0.8,0.837232
20,10,0.1,0.9,0.832175
21,100,0.1,0.7,0.830508
24,1000,0.1,0.7,0.830452
23,100,0.1,0.9,0.828814
15,1000,0.01,0.7,0.828785
16,1000,0.01,0.8,0.828729
17,1000,0.01,0.9,0.827147
25,1000,0.1,0.8,0.827119
18,10,0.1,0.7,0.827090


Save the best trained model with the best score for further testing.

In [42]:
best_mlp_model = grid_search.best_estimator_
dump(best_mlp_model, 'best_mlp_model.joblib')

['best_mlp_model.joblib']